# English Language LEarning Ability Prediction Model for Children

## Summary

--TO BE ADDED  the PROJECT SUMMARY INTRO PART

## Introduction 

### Background

-- TO BE ADDED provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your report

### Research Question

Can we predict an individual's English proficiency score based on factors such as age, education, and language background?

### Dataset

identify and describe the dataset that was used to answer the question

# Loading Packages and Functions

In [1]:
import pandas as pd
import numpy as num
import altair as alt
import vegafusion
import matplotlib
from sklearn.model_selection import train_test_split

# Reading Data

In [3]:
# Reading the dataset
dataset = pd.read_csv("https://osf.io/download/g72pq/", sep=',', on_bad_lines='skip', low_memory=False)
dataset.head()

,Unnamed: 0,id,date,time,gender,age,natlangs,primelangs,dyslexia,psychiatric,...,q34_6,q34_8,q35_1,q35_2,q35_4,q35_5,q35_7,q35_8,correct,elogit
0,1,2.0,2014-05-29,01:00:55,male,31,English,English,0,0,...,1,1,1,1,1,1,1,1,0.978947,3.621671
1,2,3.0,2014-05-29,01:01:03,male,30,English,English,0,1,...,1,1,1,1,1,1,1,1,1.000000,5.252273
2,3,3.0,2014-05-29,01:01:03,male,30,English,English,0,1,...,1,1,1,1,1,1,1,1,1.000000,5.252273
3,4,4.0,2014-05-29,01:01:13,male,19,English,English,0,0,...,1,1,1,1,1,0,1,1,0.905263,2.208853
4,5,5.0,2014-05-29,01:01:19,male,20,"Chinese/Mandarin, English","Chinese/Mandarin, English",0,0,...,1,1,1,1,1,0,1,1,0.947368,2.800602


#### Split the Dataset into Train/Test

In [4]:
df = dataset

# Splitting the DataFrame
train_df, test_df = train_test_split(df, test_size=0.3, random_state=123)

train_df.head()

,Unnamed: 0,id,date,time,gender,age,natlangs,primelangs,dyslexia,psychiatric,...,q34_6,q34_8,q35_1,q35_2,q35_4,q35_5,q35_7,q35_8,correct,elogit
413443,419829,1021813.0,2014-06-12,08:07:12,female,35,Hungarian,"French , Hungarian",0,0,...,1,1,1,1,1,1,1,1,0.926316,2.468100
385270,391249,742479.0,2014-06-09,04:48:18,male,24,Polish,Polish,0,0,...,0,1,1,1,1,0,1,0,0.726316,0.964182
444091,450777,1067504.0,2014-06-14,07:49:34,female,29,English,"English, Japanese",0,1,...,1,1,1,1,1,1,1,1,0.947368,2.800602
21235,21473,122687.0,2014-05-29,12:55:04,male,49,Finnish,Finnish,0,0,...,1,1,0,1,1,1,1,1,0.968421,3.274446
39273,39732,162389.0,2014-05-30,03:14:23,male,23,"Chinese/Hokkien, Chinese/Mandarin, English","Chinese/Hokkien, English",0,0,...,0,0,0,0,0,0,0,1,0.768421,1.183770


### EDA

## Handling the NaN values

In [5]:
# number of NaN values in each column
nan_counts = train_df.isna().sum()
print(nan_counts)

gender                       0
age                          0
natlangs                  5532
primelangs                5400
dyslexia                     0
psychiatric                  0
education                    0
tests                        0
Eng_start                    0
Eng_country_yrs         202988
house_Eng               202988
dictionary                   0
already_participated         0
countries                20753
currcountry              23770
US_region               350408
UK_region               400282
Can_region              434161
Ebonics                 350489
Ir_region               460322
UK_constituency         403298
nat_Eng                      0
prime_Eng                    0
speaker_cat                  0
type                    190365
Lived_Eng_per           210926
Eng_little               77855
correct                      0
dtype: int64


In [6]:
# Replacing NaN values for the numeric columns with zero
df_numeric_filled = train_df.select_dtypes(include='number').fillna(0)
train_df.update(df_numeric_filled)

# Dropping the other columns with plenty on NaN values (except natlangs and primelangs)
exclude_columns = ['natlangs', 'primelangs']
columns_to_drop = train_df.columns[train_df.isna().any() & ~train_df.columns.isin(exclude_columns)]
train_df = train_df.drop(columns=columns_to_drop)

# dropping rows with NaN values in natlangs and primelangs columns
train_df = train_df.dropna()

train_df

,gender,age,natlangs,primelangs,dyslexia,psychiatric,education,tests,Eng_start,Eng_country_yrs,house_Eng,dictionary,already_participated,nat_Eng,prime_Eng,speaker_cat,Lived_Eng_per,correct
413443,female,35,Hungarian,"French , Hungarian",0,0,Some Undergrad (higher ed),",WhichEnglish",8,0.0,0.0,0,0,0,0,late,0.0,0.926316
385270,male,24,Polish,Polish,0,0,Graduate Degree,",WhichEnglish",16,0.0,0.0,0,0,0,0,late,0.0,0.726316
444091,female,29,English,"English, Japanese",0,1,Undergraduate Degree (3-5 years higher ed),",WhichEnglish",0,0.0,0.0,0,0,1,1,native,0.0,0.947368
21235,male,49,Finnish,Finnish,0,0,Undergraduate Degree (3-5 years higher ed),",WhichEnglish",9,0.0,0.0,0,0,0,0,late,0.0,0.968421
39273,male,23,"Chinese/Hokkien, Chinese/Mandarin, English","Chinese/Hokkien, English",0,0,Graduate Degree,",WhichEnglish",0,0.0,0.0,0,0,1,1,native,0.0,0.768421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194278,female,18,Hungarian,English,0,0,Haven't Finished High School (less than 13 yea...,",WhichEnglish",4,0.0,0.0,0,0,0,0,foreign,0.0,0.947368
192476,male,54,"German, Hungarian",Hungarian,0,0,Graduate Degree,",WhichEnglish",15,0.0,0.0,0,0,0,0,late,0.0,0.873684
17730,female,18,English,English,0,0,Some Undergrad (higher ed),",WhichEnglish",0,0.0,0.0,0,0,1,1,native,0.0,0.947368
28030,female,24,Finnish,Finnish,0,0,Some Undergrad (higher ed),",WhichEnglish",8,0.0,0.0,0,0,0,0,late,0.0,0.873684


In [7]:
# number of NaN values in each column
nan_counts = train_df.isna().sum()
print(nan_counts)

gender                  0
age                     0
natlangs                0
primelangs              0
dyslexia                0
psychiatric             0
education               0
tests                   0
Eng_start               0
Eng_country_yrs         0
house_Eng               0
dictionary              0
already_participated    0
nat_Eng                 0
prime_Eng               0
speaker_cat             0
Lived_Eng_per           0
correct                 0
dtype: int64


In [8]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 463116 entries, 413443 to 277869
Data columns (total 18 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   gender                463116 non-null  object 
 1   age                   463116 non-null  int64  
 2   natlangs              463116 non-null  object 
 3   primelangs            463116 non-null  object 
 4   dyslexia              463116 non-null  int64  
 5   psychiatric           463116 non-null  int64  
 6   education             463116 non-null  object 
 7   tests                 463116 non-null  object 
 8   Eng_start             463116 non-null  int64  
 9   Eng_country_yrs       463116 non-null  float64
 10  house_Eng             463116 non-null  float64
 11  dictionary            463116 non-null  int64  
 12  already_participated  463116 non-null  int64  
 13  nat_Eng               463116 non-null  int64  
 14  prime_Eng             463116 non-null  int64  
 15  

In [9]:
train_df.describe()

,age,dyslexia,psychiatric,Eng_start,Eng_country_yrs,house_Eng,dictionary,already_participated,nat_Eng,prime_Eng,Lived_Eng_per,correct
count,463116.000000,463116.0,463116.000000,463116.000000,463116.000000,463116.000000,463116.0,463116.0,463116.000000,463116.000000,463116.000000,463116.000000
mean,29.981197,0.0,0.030940,5.138367,2.503055,0.109484,0.0,0.0,0.441565,0.439978,0.080023,0.937807
std,11.263265,0.0,0.173156,5.694199,7.973296,0.312246,0.0,0.0,0.496574,0.496385,0.233654,0.060923
min,7.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.073684
25%,22.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.915789
50%,27.000000,0.0,0.000000,4.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.957895
75%,35.000000,0.0,0.000000,10.000000,0.000000,0.000000,0.0,0.0,1.000000,1.000000,0.000000,0.978947
max,89.000000,0.0,1.000000,74.000000,87.000000,1.000000,0.0,0.0,1.000000,1.000000,1.100000,1.000000


# EDA


## Column data types and first few rows

In [10]:
data_types = train_df.dtypes 
first_few_rows = train_df.head()  

print(data_types)
first_few_rows

gender                   object
age                       int64
natlangs                 object
primelangs               object
dyslexia                  int64
psychiatric               int64
education                object
tests                    object
Eng_start                 int64
Eng_country_yrs         float64
house_Eng               float64
dictionary                int64
already_participated      int64
nat_Eng                   int64
prime_Eng                 int64
speaker_cat              object
Lived_Eng_per           float64
correct                 float64
dtype: object


,gender,age,natlangs,primelangs,dyslexia,psychiatric,education,tests,Eng_start,Eng_country_yrs,house_Eng,dictionary,already_participated,nat_Eng,prime_Eng,speaker_cat,Lived_Eng_per,correct
413443,female,35,Hungarian,"French , Hungarian",0,0,Some Undergrad (higher ed),",WhichEnglish",8,0.0,0.0,0,0,0,0,late,0.0,0.926316
385270,male,24,Polish,Polish,0,0,Graduate Degree,",WhichEnglish",16,0.0,0.0,0,0,0,0,late,0.0,0.726316
444091,female,29,English,"English, Japanese",0,1,Undergraduate Degree (3-5 years higher ed),",WhichEnglish",0,0.0,0.0,0,0,1,1,native,0.0,0.947368
21235,male,49,Finnish,Finnish,0,0,Undergraduate Degree (3-5 years higher ed),",WhichEnglish",9,0.0,0.0,0,0,0,0,late,0.0,0.968421
39273,male,23,"Chinese/Hokkien, Chinese/Mandarin, English","Chinese/Hokkien, English",0,0,Graduate Degree,",WhichEnglish",0,0.0,0.0,0,0,1,1,native,0.0,0.768421


## Distributions of Numeric Columns

In [11]:
alt.data_transformers.enable('vegafusion')

numeric_cols = train_df.select_dtypes(include='number').columns.tolist()

numeric_cols_dist = alt.Chart(train_df).mark_bar().encode(
    x=alt.X(alt.repeat("repeat"), type="quantitative"),
    y="count()"
).properties(
    width=250,
    height=150
).repeat(
    repeat=numeric_cols,
    columns=3
)

numeric_cols_dist

alt.RepeatChart(...)

## Correlation matrix

In [13]:
train_df.select_dtypes("number").corr('spearman').style.background_gradient()

/Users/atabak/miniconda3/envs/571/lib/python3.10/site-packages/pandas/io/formats/style.py:3819: RuntimeWarning: All-NaN slice encountered
  smin = np.nanmin(gmap) if vmin is None else vmin
/Users/atabak/miniconda3/envs/571/lib/python3.10/site-packages/pandas/io/formats/style.py:3820: RuntimeWarning: All-NaN slice encountered
  smax = np.nanmax(gmap) if vmax is None else vmax


,age,dyslexia,psychiatric,Eng_start,Eng_country_yrs,house_Eng,dictionary,already_participated,nat_Eng,prime_Eng,Lived_Eng_per,correct
age,1.000000,nan,-0.033704,0.048864,0.087302,0.044314,nan,nan,0.014649,0.013521,0.094890,0.098045
dyslexia,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
psychiatric,-0.033704,nan,1.000000,0.003635,-0.012675,-0.001150,nan,nan,0.005093,0.005063,-0.012395,-0.086274
Eng_start,0.048864,nan,0.003635,1.000000,0.214736,0.065382,nan,nan,-0.818232,-0.815355,0.219387,-0.465141
Eng_country_yrs,0.087302,nan,-0.012675,0.214736,1.000000,0.570033,nan,nan,-0.387159,-0.385664,0.936418,-0.001565
house_Eng,0.044314,nan,-0.001150,0.065382,0.570033,1.000000,nan,nan,-0.248430,-0.247403,0.511176,0.024275
dictionary,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
already_participated,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
nat_Eng,0.014649,nan,0.005093,-0.818232,-0.387159,-0.248430,nan,nan,1.000000,0.996786,-0.376576,0.407712
prime_Eng,0.013521,nan,0.005063,-0.815355,-0.385664,-0.247403,nan,nan,0.996786,1.000000,-0.375156,0.406097


## Dropping the Columns

In [ ]:
# Creating a list of columns related to questions to drop
columns_to_drop = [col for col in dataset.columns if col.startswith('q')]

# Dropping the question columns from the DataFrame
dataset = dataset.drop(columns_to_drop, axis=1)

# Dropping the irrelevant columns from the DataFrame
dataset = dataset.drop("elogit", axis=1)
df = dataset.drop("tests", axis=1)
dataset = dataset.iloc[:,4:]